In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (/home/sriteja/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import pickle
# with open('yelp_dataset.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [3]:
with open('yelp_dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

/home/sriteja/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [3]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data = train_dataset['text']
train_labels = train_dataset['label']
test_data = test_dataset['text']
test_labels = test_dataset['label']

# Convert labels to lists (optional)
# train_labels = train_labels.tolist()
# test_labels = test_labels.tolist()

In [4]:
print(train_data[0])
print(train_labels[0])


dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
4


## Preprocess

In [7]:
import numpy as np

train_tokens = []
with open('yelp_train_tokens_filtered.pkl', 'rb') as file:
    train_tokens = pickle.load(file)

test_tokens = []
with open('yelp_test_tokens_filtered.pkl', 'rb') as file:
    test_tokens = pickle.load(file)
    
print("Loaded tokens")



Loaded tokens


In [8]:
# in train data replace every 1000th word with UNK randomly

import random

for i in range(len(train_tokens)):
    for j in range(len(train_tokens[i])):
        if random.randint(1, 1000) == 1:
            train_tokens[i][j] = 'UNK'
        

In [9]:
# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab = set()
for tokens in train_tokens:
    vocab.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict = {}
for i, word in enumerate(vocab):
    vocab_dict[word] = i

print(len(vocab_dict))
# Initialize BoW matrices for training and testing data


45287


In [10]:
reduced_train_tokens = train_tokens[:20000]

# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab_reduced = set()
for tokens in reduced_train_tokens:
    vocab_reduced.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict_reduced = {}
for i, word in enumerate(vocab_reduced):
    vocab_dict_reduced[word] = i

print(len(vocab_dict_reduced))

train_bow = np.zeros((len(reduced_train_tokens), len(vocab_reduced)))

print("bow train")
# Convert text to BoW vectors
for i, tokens in enumerate(reduced_train_tokens):
    if i % 5000 == 0:
        print(i)
    for token in tokens:
        train_bow[i][vocab_dict_reduced[token]] += 1




27982
bow train
0
5000
10000
15000


In [11]:
reduced_test_tokens = test_tokens[:5000]
test_bow = np.zeros((len(reduced_test_tokens), len(vocab_reduced)))

print("bow test")

for i, tokens in enumerate(reduced_test_tokens):
    if i % 1000 == 0:
        print(i)
    for token in tokens:
        if token in vocab_dict_reduced:
            test_bow[i][vocab_dict_reduced[token]] += 1
        else:    # if there is an unknown word, add it to the UNK column 
            test_bow[i][vocab_dict_reduced['UNK']] += 1

bow test
0
1000
2000
3000
4000


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Logistic Regression classifier
classifier = LogisticRegression(max_iter=200)

# Fit the classifier to the training data with a progress bar
# with tqdm(total=len(train_bow[:20000])) as pbar:
# classifier.fit(train_bow[:20000], train_labels[:20000])
    # pbar.update(len(train_bow[:20000]))

# Predict the labels of the test data with a progress bar
# with tqdm(total=len(test_bow[:5000])) as pbar:
# y_pred = classifier.predict(test_bow[:5000])
    # pbar.update(len(test_bow[:5000]))

# Calculate accuracy and other classification metrics
# print('Accuracy: {}'.format(accuracy_score(test_labels[:5000], y_pred)))
# print(classification_report(test_labels[:5000], y_pred))

# save the classifier
with open('yelp_classifier_log_reg_06_200itr.pkl', 'wb') as file:
    pickle.dump(classifier, file)

/home/sriteja/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.497
              precision    recall  f1-score   support

           0       0.68      0.66      0.67      1141
           1       0.41      0.42      0.41      1009
           2       0.40      0.40      0.40      1003
           3       0.43      0.40      0.42       986
           4       0.56      0.59      0.57       861

    accuracy                           0.50      5000
   macro avg       0.49      0.49      0.49      5000
weighted avg       0.50      0.50      0.50      5000



In [28]:
X_train = train_bow[:17000]
y_train = train_labels[:17000]
X_val = train_bow[17000:20000, :]
y_val = train_labels[17000:20000]
X_test = test_bow[:5000]
y_test = test_labels[:5000]

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

### Wandb

In [13]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: silvertongue1729. Use `wandb login --relogin` to force relogin


True

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val accuracy',
        'goal': 'maximize'   
    },
    'parameters': {
        'max_iter': {
            'values': [500]
        },
    },
    "name": "bow_yelp_sweep"
}

In [14]:
if wandb.run is not None:
    wandb.finish()

wandb.init(project="SMAI_project_bow", config={
    "model": "BoW", "dataset": "yelp review", "max_iter": 500
})

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

classifier = LogisticRegression(max_iter=500)

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_train)
y_pred_val = classifier.predict(X_val)

print(y_pred.shape)
print(y_pred_val.shape)

print(y_train.shape)
print(y_val.shape)

train_accuracy = accuracy_score(y_train, y_pred)
val_accuracy = accuracy_score(y_val, y_pred_val)

# Calculate accuracy and other classification metrics
print('Val Accuracy: {}'.format(val_accuracy))
print(classification_report(y_val, y_pred_val))

wandb.log({"val accuracy": val_accuracy, "train accuracy": train_accuracy})


ValueError: Found array with 0 sample(s) (shape=(0, 27982)) while a minimum of 1 is required by LogisticRegression.

In [29]:
print(X_train.shape)
print(X_val.shape)
# print(y_train.shape)

(17000, 27982)
(3000, 27982)


(17000,)
(3000,)
(17000,)
(3000,)
Val Accuracy: 0.9606666666666667
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       621
           1       0.98      0.96      0.97       552
           2       0.95      0.95      0.95       602
           3       0.95      0.93      0.94       612
           4       0.95      0.97      0.96       613

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000



In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

def train(config=None):
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        # Initialize the Logistic Regression classifier
        classifier = LogisticRegression(C=config.C, solver=config.solver, max_iter=config.max_iter)

        # Fit the classifier to the training data with a progress bar
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_val)

        # Calculate accuracy and other classification metrics
        print('Accuracy: {}'.format(accuracy_score(y_val, y_pred)))
        print(classification_report(y_val, y_pred))
        
        wandb.log({"val accuracy": accuracy_score(y_val, y_pred), "train accuracy": accuracy_score(y_train, classifier.predict(X_train))})

In [ ]:
# wandb.agent(sweep_id, train, count=8)